In [ ]:
import sklearn
import pyarrow
import glob
import tqdm

import numpy as np
import awkward as ak
import mplhep as hep
import matplotlib.pyplot as plt
plt.style.use(hep.style.CMS)

from sklearn.metrics import roc_curve
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit

In [ ]:
particle_types = {
    'chhad': 1,
    'nhad': 2,
    'HFem': 3,
    'HFhad': 4,
    'gamma': 5,
    'e': 6,
    'mu': 7,
}

particle_types_cmssw = {
    'chhad': 1,
    'e': 2,
    'mu': 3,
    'gamma': 22,
    'nhad': 130,
    'HFhad': 1,
    'HFem': 2,
}

particle_types_literal = {
    'chhad': 'ch. had.',
    'nhad': 'n. had.',
    'e': r'$e^\pm$',
    'mu': r'$\mu^\pm$',
    'gamma': r'$\gamma$',
    'HFhad': 'HF had.',
    'HFem': 'HF e.m.',
}

types = ['nhad', 'HFem', 'HFhad', 'gamma']
particle_types_nums = [1, 2, 22, 130]

eta_bins = [0.0, 2.5, 5] 

TPR_CUT = 0.8
ETA_CUT = eta_bins[1]

In [ ]:
def exact_threshold_for_tpr(y_true, y_scores, desired_tpr):
    """
    Finds the exact threshold corresponding to the desired TPR via interpolation.
    
    Parameters:
        y_true (array-like): Binary ground truth labels.
        y_scores (array-like): Classifier scores (e.g., probabilities).
        desired_tpr (float): Desired true positive rate (0 < TPR < 1).
    
    Returns:
        float: Threshold interpolated to give the desired TPR.
    """
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)

    if desired_tpr < tpr.min() or desired_tpr > tpr.max():
        raise ValueError(f"Desired TPR {desired_tpr:.4f} is outside the achievable range: "
                         f"{tpr.min():.4f}–{tpr.max():.4f}")

    # Interpolate threshold as a function of TPR
    threshold_interp = interp1d(tpr, thresholds, kind='linear', bounds_error=True)
    threshold = threshold_interp(desired_tpr)
    return float(threshold)

In [ ]:
df = ak.concatenate([ak.from_parquet(_, columns=['particles']) for _ in tqdm.tqdm(glob.glob('/isilon/hadoop/store/user/kaho/cms_pf_qcd/*parquet'))])

In [ ]:
for type_ in types:

    selection = (df.particles.pred.cls_id==particle_types[type_]) 
    pt_reco = ak.flatten(df.particles.pred[selection].pt)
    eta_reco = ak.flatten(df.particles.pred[selection].eta)
    
    plt.figure()
    plt.hist(pt_reco[abs(eta_reco) < ETA_CUT], bins=100, histtype='step', label=particle_types_literal[type_]+r', |$\eta$| < '+str(ETA_CUT))
    plt.hist(pt_reco[abs(eta_reco) > ETA_CUT], bins=100, histtype='step', label=particle_types_literal[type_]+r', |$\eta$| > '+str(ETA_CUT))
    plt.yscale('log')


    plt.legend()
    plt.xlabel('$p_{T}$ [GeV]', loc='right')
    plt.ylabel('Particles', loc='top')

    hep.cms.label("Preliminary", data=False, com=14, year='Run 3')

    plt.tight_layout()
    plt.show()


In [ ]:
thresholds = {}
x_bins = {}
for type_ in types:
    print(type_)
    
    thresholds[type_] = []
    x_bins[type_] = []
    
    if type_=='HFem':
        pt_bins = [[[0, 5], [5, 8], [8, 20]]]
    elif type_=='HFhad':
         pt_bins = [[[0, 5], [5, 10], [10, 20], [20, 30], [30, 50]]]
    elif type_=='gamma':
        pt_bins = [[[0, 5], [5, 10], [10, 15], [15, 30], [30, 50], [50, 100], [100, 200], [200, 400]], 
                   [[0, 5], [5, 10], [10, 15], [15, 40], [40, 100]]]
    else:
        pt_bins = [[[0, 5], [5, 10], [10, 15], [15, 20], [20, 30], [30, 50], [50, 100], [100, 200]],
                   [[0, 5], [5, 10], [10, 15], [15, 20], [20, 30], [30, 40], [40, 100]]]
        
    for eta_bin, pt_bin_ in enumerate(pt_bins): 
        print('eta bin', eta_bin)
        thresholds[type_].append([])
        x_bins[type_].append([])
        for pt_bin in pt_bin_:
            print('pt edges', pt_bin)
            pt_lo, pt_hi = pt_bin[0], pt_bin[1]
            selection = (df.particles.pred.cls_id==particle_types[type_]) & (df.particles.pred.pt<pt_hi) & (df.particles.pred.pt>pt_lo) 
            
            truth = ak.flatten((df.particles.target[selection].ispu!=1) + 0)
            pred = ak.flatten(df.particles.pred[selection].ispu)
            eta = abs(ak.flatten(df.particles.pred[selection].eta))
            pred_pt = abs(ak.flatten(df.particles.pred[selection].pt))
            
            # if pt_hi > 1000:
            #     pt_hi = ak.max(pred_pt)
    
            x_bins[type_][-1].append((pt_hi+pt_lo)/2)
            if 'HF' in type_:
                thresholds[type_][-1].append(1-exact_threshold_for_tpr(truth, 1-pred, 0.8))
            else:
                if eta_bin==0:
                    thresholds[type_][-1].append(1-exact_threshold_for_tpr(truth[eta < ETA_CUT], 1-pred[eta < ETA_CUT], TPR_CUT))
                else:
                    thresholds[type_][-1].append(1-exact_threshold_for_tpr(truth[eta > ETA_CUT], 1-pred[eta > ETA_CUT], TPR_CUT))

In [ ]:
def exp_func(x, a, b, c):
    return a * np.exp(-b * x) + c

def linear_func(x, a, b):
    return a * x + b

formulas = {}
    
for type_ in types:
    useLogx = False
    thresholds_type = thresholds[type_]
    
    label = type_+r', |$\eta$| < '+str(ETA_CUT) if not 'HF' in type_ else type_
    plt.scatter(x_bins[type_][0], thresholds_type[0], label=label)

    model_func = exp_func
    p0s = (1, 0.1, 0.1)

    if 'HF' in type_:
        params, covariance = curve_fit(model_func, x_bins[type_][0], thresholds_type[0], p0=p0s) 
        x_fit = np.linspace(min(x_bins[type_][0]), max(x_bins[type_][0]), 200)
        y_fit = model_func(x_fit, *params)
        useLogx = False
    else:
        params, covariance = curve_fit(model_func, np.log(x_bins[type_][0]), thresholds_type[0], p0=p0s)
        x_fit = np.linspace(min(x_bins[type_][0]), max(x_bins[type_][0]), 200)
        y_fit = model_func(np.log(x_fit), *params)
        useLogx = True
        
    a_fit, b_fit, c_fit = params

    if useLogx:
        eq = f'{a_fit} * exp(-{b_fit} * log(x)) + {c_fit}'
        eq_display = f'{a_fit: .2f} * exp(-{b_fit: .2f} * log(x)) + {c_fit: .2f}'
    else:
        eq = f'{a_fit} * exp(-{b_fit} * x) + {c_fit}'
        eq_display = f'{a_fit:.2f} * exp(-{b_fit:.2f} * x) + {c_fit:.2f}'

    formulas[(particle_types_cmssw[type_], 0)] = eq
    formulas[(particle_types_cmssw[type_], 1)] = eq
    
    plt.plot(x_fit, y_fit, label='Fit')

    if not 'HF' in type_:
        plt.scatter(x_bins[type_][1], thresholds_type[1], label=type_+r', |$\eta$| > '+str(ETA_CUT))
        params, covariance = curve_fit(model_func, x_bins[type_][1], thresholds_type[1], p0=p0s)  # p0 = initial guess    
        x_fit = np.linspace(min(x_bins[type_][1]), max(x_bins[type_][1]), 200)
        y_fit = model_func(x_fit, *params)
        plt.plot(x_fit, y_fit, label='Fit')
        
        if useLogx:
            eq = f'{a_fit} * exp(-{b_fit} * log(x)) + {c_fit}'
        else:
            eq = f'{a_fit} * exp(-{b_fit} * x) + {c_fit}'

        formulas[(particle_types_cmssw[type_], 1)] = eq

    plt.ylabel("MLPF-PU classifier value at TPR = 0.8", loc='top')
    plt.xlabel("$p_{T}$ [GeV]", loc='right')
    hep.cms.label("Preliminary", data=False, com=14, year='Run 3')
    plt.legend()

    plt.show()

In [ ]:
formulas.keys(), formulas

In [ ]:
import correctionlib.schemav2 as cs
import json

contents = []
for i in range(len(eta_bins) - 1):
    cat = cs.Category(
        nodetype="category",
        input="particle_type",
        content=[
            {
                "key": ptype,
                "value": cs.Formula(
                    nodetype="formula",
                    expression=formulas[(ptype, i)],
                    parser="TFormula",
                    variables=["pt"]
                )
            }
            for ptype in particle_types_nums
        ]
    )
    contents.append(cat)

# Build the correction object
correction = cs.Correction(
    name="80TPR",
    description="Correction depending on particle_type (int), pt, and abs eta",
    version=1,
    inputs=[
        {"name": "abseta", "type": "real"},
        {"name": "particle_type", "type": "int"},
        {"name": "pt", "type": "real"},
    ],
    output={"name": "weight", "type": "real"},
    data=cs.MultiBinning(
        nodetype="multibinning",
        inputs=["abseta"],
        edges=[eta_bins],
        content=contents,
        flow='clamp'
    ),
)

# Wrap and export
cset = cs.CorrectionSet(
    schema_version=2,
    corrections=[correction],
)

with open("mlpfpu_threshold_80TPR.json", "w") as f:
    json.dump(cset.model_dump(), f, indent=2)

print("✅ CorrectionLib JSON written as 'mlpfpu_threshold_80TPR.json'")


In [ ]:
!correction summary mlpfpu_threshold_80TPR.json

In [ ]:
from correctionlib import CorrectionSet
test = CorrectionSet.from_file(f'mlpfpu_threshold_80TPR.json')
test['80TPR'].evaluate([0.2, 2.8], [4, 7], [200.2, 3])